<a href="https://colab.research.google.com/github/clarakl/UoA-GEOG761/blob/main/761_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4: Deep Learning for Satellite Data
This notebook uses a pre-made training dataset to develop a simple deep learning classifier for finding solar pannels in Sentinel 2 imagery. The learning objectives for this lab are to:

1. Load and use a pre-labeled training dataset (e.g. polygons or patches with land cover classes).
2. Prepare that data for use in a deep learning model.
3. Train a deep learning model in TensorFlow/Keras using these labels.

At the end of the lab you will be asked to reflect on model performance, generalization, and sources of error.


In [ ]:
# Set up, couple of new libraries here so expect install times to take longer than you have been used to
!pip install -q tensorflow rasterio geemap matplotlib scikit-learn tqdm --quiet

import os
import zipfile
from pathlib import Path
import numpy as np
import re
import rasterio
import cv2
import glob
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
# Set our random seeds for reproducibility
SEED = 42 #<- the meaning of life, this is why you always see people setting seeds to 42 by the way...
np.random.seed(SEED)
tf.random.set_seed(SEED)

Your training data has been downloaded from:

*   [Solafune](https://solafune.com/competitions/5dfc315c-1b24-4573-804f-7de8d707cd90?id=&menu=data&topicId=54728653-0e25-4975-baad-6fe2f5185844)

Find the data on Canvas or download directly from their website the 'train.zip'. The competition that this dataset was created for is now over, but that is good for us as it means you can look at the winning solutions for inspiration!

If you want to download the data yourself you will need to sign up for an account and then 'enter' the competition. You can do this all without charge but it does require an email address sign up.

We are just going to use the 'train.zip' data and split that into our train and test, in order to reduce the burden on our notebooks. Remember that ultimately we would want to validate it, and you will see that the linked data source actually has a completely seperate store of images for that purpose.

Take that .zip file and get it into your g-drive/local space as per usual practice. Leave it zipped.


In [ ]:
# Assume train.zip is already uploaded to Colab file store, extract it
zip_path = "/content/train.zip"
extract_dir = Path("/content/train_data")
extract_dir.mkdir(parents=True, exist_ok=True)


with zipfile.ZipFile(zip_path, 'r') as z:
  z.extractall(extract_dir)

With our data extracted, we now need to build lists of each sample patch and the labelled mask of it, connecting the two. I have written this relatively robustly (e.g. you can set it to handle different file extensions and it checks that it is finding the correct files), so that hopefully you can use it more widely than just in this lab.

In [ ]:
# Set the file extenion we are using
f_ex = "*.tif"

# Locate directories
s2_dir = next((p for p in extract_dir.rglob('*') if p.is_dir() and 's2_image' in p.name.lower()), None)
mask_dir = next((p for p in extract_dir.rglob('*') if p.is_dir() and 'mask' in p.name.lower()), None)

assert s2_dir and mask_dir, "Could not locate s2_image/ and mask/ directories."
print("Found Sentinel-2 image dir:", s2_dir)
print("Found mask dir:", mask_dir)

# Match files to each other from each sub-directory by trailing numeric ID
s2_files = {re.search(r'_(\d+)\.tif$', f.name).group(1): f for f in Path(s2_dir).glob(f_ex)}
mask_files = {re.search(r'_(\d+)\.tif$', f.name).group(1): f for f in Path(mask_dir).glob(f_ex)}

# Build pairs of images and masks
common_ids = sorted(set(s2_files.keys()) & set(mask_files.keys()))
assert len(common_ids) > 0, "No paired s2_image/mask files found with matching IDs."

pairs = []
for i in common_ids:
    s2_path, mask_path = s2_files[i], mask_files[i]
    with rasterio.open(s2_path) as s2, rasterio.open(mask_path) as m:
        assert (s2.height, s2.width) == (m.height, m.width), \
            f"Shape mismatch: {s2_path.name} vs {mask_path.name}"
    pairs.append((s2_path, mask_path))

print(f"Found {len(pairs)} valid paired samples.")
print("Example pair:", pairs[0])

In [ ]:
# Quick sanity check: visualize one random pair out of the first 31 available
# Run this cell a few times to check that you have different files and that the mask looks about right
pair_num = np.random.randint(0,30)
s2_path, mask_path = pairs[pair_num]

with rasterio.open(s2_path) as src:
    s2_img = src.read([3,2,1])  # RGB bands for Sentinel-2 (B4, B3, B2)
    s2_img = np.transpose(s2_img, (1,2,0))
    s2_img = (s2_img - s2_img.min()) / (s2_img.max() - s2_img.min()) # min/max normalization to have the lowest value be 0 and the highest be 1 to use the full range, if we would not substract the min we would lose some range

with rasterio.open(mask_path) as src:
    mask = src.read(1)

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(s2_img)
plt.title("Sentinel-2 RGB Patch")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(mask, cmap="gray")
plt.title("Mask")
plt.axis("off")
plt.show()

Next up we are going to prepare our training data into numpy arrays that are then able to be passed into TensorFlow. However, if we try to hold all 2000+ patches in RAM whilst we do, the Colab instance will crash. If on your own laptop, it will also die unless you have spent a lot more money than I have on mine. Furthermore, even if we had a HPC level of RAM available it is a good idea to do things in a 'memory safe' manner that means we can scale our training with more samples if we so desired it.

The safe pattern is therefore to:
1. Load a batch of patches.
2. Normalize/resize them.
3. Save them to disk (e.g. as .npy, .npz, or TFRecords/HDF5).
4. Free memory and move on.
5. Then later, when training, you stream batches directly from disk.

Here’s a disk-based pipeline with batching using numpy.savez_compressed (safe + easy to reload):

In [ ]:
def resize_or_pad(arr, target_h, target_w):
    h, w = arr.shape[:2]
    c = arr.shape[2] if arr.ndim == 3 else 1

    arr = cv2.resize(
        arr,
        (target_w, target_h),
        interpolation=cv2.INTER_NEAREST if c == 1 else cv2.INTER_LINEAR)

    if arr.ndim == 2:
        arr = arr[..., np.newaxis]
    return arr


# Set storage folder
OUTPUT_DIR = "patch_store"
os.makedirs(OUTPUT_DIR, exist_ok=True)

TARGET_HEIGHT, TARGET_WIDTH = 256, 256 # Tune based on memory available
BATCH_SIZE = 100  # Tune based on memory available


X_batch, Y_batch = [], []
batch_idx = 0

for i, (s2_path, mask_path) in enumerate(tqdm(pairs, desc="Processing")):
    # tqdm is positioned so that it wraps the iterable in the for loop
    # Load Sentinel-2
    with rasterio.open(s2_path) as src:
        img = src.read().astype(np.float32)  # (bands, H, W)
        img = np.transpose(img, (1, 2, 0))  # (H, W, bands)

    # Load mask
    with rasterio.open(mask_path) as src:
        mask = src.read(1).astype(np.uint8)

    # Resize/pad
    img = resize_or_pad(img, TARGET_HEIGHT, TARGET_WIDTH)
    mask = resize_or_pad(mask, TARGET_HEIGHT, TARGET_WIDTH)

    # Normalize
    img = img / 10000.0

    # Add to batch
    X_batch.append(img)
    Y_batch.append(mask)

    # Save batch to disk when full
    if len(X_batch) >= BATCH_SIZE:
        X_batch = np.stack(X_batch)
        Y_batch = np.stack(Y_batch)

        np.savez_compressed(
            os.path.join(OUTPUT_DIR, f"batch_{batch_idx:04d}.npz"),
            X=X_batch, Y=Y_batch
        )

        # Reset
        batch_idx += 1
        X_batch, Y_batch = [], []

# Save any leftovers
# If the last few images don't add up to 100, you still want to save them, just not in a batch of 100 but whatever number of images is left
if len(X_batch) > 0:
    X_batch = np.stack(X_batch)
    Y_batch = np.stack(Y_batch)
    np.savez_compressed(
        os.path.join(OUTPUT_DIR, f"batch_{batch_idx:04d}.npz"),
        X=X_batch, Y=Y_batch
    )

# By the way it is the 'tqdm' element that is providing the nifty progress bar. I always like to include it when waiting for things so that I know it is actually running.

# This block might take five to ten minutes to run as we are doing this in a linear manner (rather than parallell processing).
# An optimised version of thise code is to pass each batch to a seperate CPU thread/core.

5-10 minutes later... (mine took 8 minutes and 32 seconds), we can now move on. Because we are now not storing our training data in our RAM like we did in the lecture exercise, we have to load it in as we want it.

First up, set a bunch of parameters that we can modify in order to save our RAM load. In short, the lower these values the less RAM you will use but the worse the results:

In [ ]:
# Paramaters for RAM saving
TARGET_HEIGHT, TARGET_WIDTH = 128, 128  # smaller patch -> saves RAM & compute
BANDS = 3  # RGB only -> reduces memory
BATCH_SIZE = 4  # small batch -> reduces RAM usage
MAX_TRAIN_FILES = 5  # only use subset for lab demonstration. this is the number of batches, so 5 x 100 patches
MAX_VAL_FILES = 2

**Q1**: We have used three bands here. What is the total numnber of bands we could make available to the CNN if we so chose?

A1: Sentinel-2 has a total of 13 bands, which are also included in our data as written in the description of the competition on Solafune: 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12'.

Next we find our patch files:

In [ ]:
# Set our random seeds for reproducibility
SEED = 42 #<- the meaning of life, this is why you always see people setting seeds to 42 by the way...
np.random.seed(SEED)
tf.random.set_seed(SEED)
#Putting this here again because everytime I rerun the random shuffle, it continues where it left off with the 42 seed but that does not mean that we get the same shuffling again.

# Locate the batched .npz files
all_files = sorted(glob.glob("patch_store/*.npz"))
np.random.shuffle(all_files)

train_files = all_files[:MAX_TRAIN_FILES]
val_files   = all_files[-MAX_VAL_FILES:]

In [ ]:
def count_total_patches(file_list):
    """
    Calculates the total number of patches by summing the
    patches in each .npz file in the list.
    """
    total_patches = 0
    print("--- Counting patches in the following files: ---")
    for file_path in file_list:
        with np.load(file_path) as data:
            num_patches_in_file = data['X'].shape[0]
            print(f"File: {file_path.split('/')[-1]} -> Patches: {num_patches_in_file}")
            total_patches += num_patches_in_file
    return total_patches

# --- Calculate and print the totals ---
if len(all_files) >= (MAX_TRAIN_FILES + MAX_VAL_FILES):
    total_train_patches = count_total_patches(train_files)
    print(f"\n>>> Total training patches: {total_train_patches}\n")

    total_val_patches = count_total_patches(val_files)
    print(f"\n>>> Total validation patches: {total_val_patches}\n")
else:
    print("Not enough batch files found to create train and validation sets.")

**Q2**: What is the total number of patches in our training data and the total number in our validation set? (With the parameters set as I have done in the notebook).

Tip: you can either programmatically count what is in train_files/val_files, or you can calculate it from the parameters that have been set across the cells above this one.

A2: Well, you confused me a lot with the random shuffling that you did on the order of the batches, because I thought that we must have 500 patches in the training set (the 5 first batches x 100 patches) and 166 in the validation set (the 2 last batches, of which the last one contains the “leftovers”, and we have 2066 patches in total so this one must contain 66). But, with the shuffling, we ended up using the following batches:


---



Training data:

File: batch_0000.npz -> Patches: 100

File: batch_0017.npz -> Patches: 100

File: batch_0015.npz -> Patches: 100

File: batch_0001.npz -> Patches: 100

File: batch_0008.npz -> Patches: 100

Total training patches: 500



---


Validation data:

File: batch_0019.npz -> Patches: 100

File: batch_0006.npz -> Patches: 100

Total validation patches: 200


---



So, the last two batches were actually batch 19 and batch 6 which both did not happen to be the very last batch that was created, so both had 100.


We then need to change our masks to a binary label. If we were doing segmenation we would keep it as a mask, but in this case we are just trying to find patches that contain solar pannels rather than the specific solar pannel outline.

In [ ]:
# Mask to binary label
def mask_to_label(mask_batch):
    """1 if any solar pixel in mask, else 0"""
    labels = (np.sum(mask_batch, axis=(1,2,3)) > 0).astype(np.float32)
    return labels

Now we need a tool to load our data in from the batched files:

In [ ]:
# Data loader
def npz_loader_cls(path):
    """Load one .npz batch, downsample patches, convert to RGB, convert mask to label"""
    data = np.load(path.numpy().decode("utf-8"))
    X = data["X"]
    Y_mask = data["Y"]

    # RAM saving: pick first 3 bands only
    X = X[..., :BANDS]

    # RAM saving: downsample patches
    from skimage.transform import resize
    X_resized = np.zeros((X.shape[0], TARGET_HEIGHT, TARGET_WIDTH, BANDS), dtype=np.float32)
    for i in range(X.shape[0]):
        X_resized[i] = resize(X[i], (TARGET_HEIGHT, TARGET_WIDTH, BANDS), anti_aliasing=True)

    X = X_resized

    Y = mask_to_label(Y_mask)
    return X, Y

We can now define our training and validation datasets, which will only be actually populated with data as required, rather than loading it all into memory at once.

In [ ]:
# TensorFlow dataset pipeline
def tf_wrapper(path):
    X, Y = tf.py_function(npz_loader_cls, [path], [tf.float32, tf.float32])
    X.set_shape([None, TARGET_HEIGHT, TARGET_WIDTH, BANDS])
    Y.set_shape([None])
    # Flatten file-batch into individual samples
    return tf.data.Dataset.from_tensor_slices((X, Y))

def make_cls_dataset(file_list, batch_size=BATCH_SIZE, shuffle=True, repeat=True):
    files = tf.data.Dataset.from_tensor_slices(file_list)
    if shuffle:
        files = files.shuffle(len(file_list))

    # RAM saving: stream samples via interleave
    ds = files.interleave(
        lambda f: tf_wrapper(f),
        cycle_length=4,
        num_parallel_calls=tf.data.AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(2048)

    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    if repeat:
        ds = ds.repeat()

    return ds

train_ds = make_cls_dataset(train_files, batch_size=BATCH_SIZE, shuffle=True)
val_ds   = make_cls_dataset(val_files, batch_size=BATCH_SIZE, shuffle=False, repeat=False)

**Q3**: Are we shuffling the training dataset? Are we shuffling the validation dataset?

A3: We are shuffling the training dataset but explicitly disabling it for the validation dataset.
Hence training yes, validation no.
This is common machine learning practice, a) to avoid the model to learn the order of examples and b) to make sure that the validation dataset stays a consistent benchmark to measure the true progress of the model.


Finally, we need to specify our model settings. We are just using a simple CNN here, the same sort as we used the lecture exercise. But, this time it has signficantly more layers and therefore parameters (but far less than you might use for your project). It may take a few minutes to set up:

In [ ]:
# We are determining the input shape from one batch first
for X_sample, _ in train_ds.take(1):
    input_shape = X_sample.shape[1:]  # (H, W, bands)
    break

# Define a simple CNN
inputs = layers.Input(shape=input_shape)
x = layers.Conv2D(16, 3, activation='relu', padding='same')(inputs)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(32, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs, outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

**Q4**: Which type of activation function is our final layer using and why are we using this function as our final layer?

A4: We are using sigmoid in our final layer and relu in the hidden layers, which is apparently a common practice. The sigmoid function is very common for binary classification, which is what we are doing here. The function outputs a value between 0 and 1, which can be interpreted as the model's confidence that the input image belongs to the positive class. I see that you have not changed the threshold, so the function will fire at 0.5 and above.


With the model ready to train, we have one more preperation step to make. We need to count the total samples available in this particular run. This is because you can change the number of samples via the RAM paramaters earlier, and therefore we should assume in this code a static number of samples will always be available when setting our number of steps per epoch.

In [ ]:
# Count the total samples
def count_samples(files):
    total = 0
    for f in files:
        total += np.load(f)["X"].shape[0]
    return total

n_train = count_samples(train_files)
n_val   = count_samples(val_files)

steps_per_epoch = max(1, n_train // BATCH_SIZE)
validation_steps = max(1, n_val // BATCH_SIZE)

Finally, we train the model! It should take 5-10 minutes to run on a free-user Colab notebook.

In [ ]:
# This 'callbacks' set of instructions is a handy way to avoid wasting time
callbacks = [
    EarlyStopping(
        monitor="val_loss",          # Watch the validation loss
        patience=3,                  # If it doesn't improve for 3 epochs, stop training
        restore_best_weights=True    # Roll back to the best-performing weights
    )]

# Run the model
history = model.fit(
    train_ds,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
    validation_steps=validation_steps,
    epochs=5,  # small number for lab demonstration
    callbacks=callbacks)

Visualizie the predictions:

In [ ]:
# Plot the training curves
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label="train")
plt.plot(history.history['val_loss'], label="val")
plt.title("Loss")
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label="train")
plt.plot(history.history['val_accuracy'], label="val")
plt.title("Accuracy")
plt.legend()
plt.show()

In [ ]:
def show_patch_predictions_grid(dataset, model, n_rows=2, n_cols=5):
    """
    Display predictions in a grid: n_rows x n_cols
    Includes histogram stretch for better visualization of RGB patches.
    """
    def stretch(img):
        """Contrast stretch to 0–1 for display."""
        img = img.astype(np.float32)
        p2, p98 = np.percentile(img, (2, 98))
        img = np.clip((img - p2) / (p98 - p2 + 1e-6), 0, 1)
        return img

    for X, Y_true in dataset.take(1):
        # live classification of the patch using the trained model
        Y_pred = model.predict(X, verbose=0)
        n_total = min(n_rows * n_cols, X.shape[0])

        fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*3, n_rows*3))
        axes = axes.flatten()

        for i in range(n_total):
            rgb = X[i, :, :, :3].numpy()
            rgb = stretch(rgb)  # apply histogram stretch
            axes[i].imshow(rgb)
            axes[i].set_title(f"T:{int(Y_true[i].numpy())}\nP:{Y_pred[i,0]:.2f}")
            axes[i].axis("off")

        # hide any unused axes
        for i in range(n_total, len(axes)):
            axes[i].axis("off")

        plt.tight_layout()
        plt.show()

# Example
show_patch_predictions_grid(val_ds, model, n_rows=1, n_cols=5)

Remembering that this is effectively still a toy CNN with tiny data, we can see that we have built a model that gives us a probability of the patches containing a solar pannel. We know that for all of the patchs used in this training, the answer is true...

**Q5**: Is this a robust and reliable model for the detection of solar panels in a new Sentinel 2 image that has never been seen by the model before? Explain your answer with reference to the design of the training data, the settings of the CNN and the performance statistics seen.

A5:
It’s quite bad I believe, even for being only a demonstration. We chose to use only 500 patches for training although we have 2066 available, and validation is even less. We should aim for something like a 80-10-10 split for training, validation and test, which would result in approximately 1652-207-207 patches respectively. So validation is alright, but we have too few training samples. I also believe that only using the visible light bands loses way too much information, as wavelengths like NIR and SWIR are very useful for distinguishing artificial materials from natural ones. I am not too sure about how badly the downsampling to a resolution of 128x128 affects the performance. I'll think about not doing that for the next question, but I am afraid that leaving the original resolution will drastically increase the computational complexity of my model.

As for the CNN architecture, I believe that more convolutional layers would give the model more possibility to learn more features instead of just basic edges or color blobs. But looking at the data that we’re working with, maybe we currently don’t need more. The number of epochs is also, excuse my language, ridiculous, but after I googled it seems like there are very simple models that work with as few as 7-10 epochs, but I believe that we should increase the number of epochs. This would also make the early stop warning with a patience of 3 epochs more meaningful.

I believe that the very flat validation accuracy of 0.9150 throughout all of the epochs is a major red flag. As you (i.e. Tom, if someone else is grading) mentioned, all of the patches have solar panels in them, so the model gets lazy and learns to only predict “yes”. The sigmoid function at the end introduces a little bit of uncertainty, because the model knows in theory that there should be a “no” class, which is why we never get to 100% accuracy. The process of overfitting also shows in the loss consistently dropping in the training dataset, while rising in the validation dataset because it learns the wrong things (loss drop in training while loss rise in validation is a typical sign of overfitting).

To fix that, I will create the “no” class and do data augmentation, because creating 2066 new patches would take me 3 days (educated guess).


**Q6**: Exercise: using the materials taught in Lab 3 and given your answer to Q5, re-design your training data so that it is a robust training set for the detection of solar panels in S2 data. Your answer needs to include:
*   A paragraph that explains what you have added to the train_data and any other steps taken in order to enhance the robustness of the training.
*   A figure that demonstrates your newly added patches vs. the existing patches (remember that it needs to be publication quality).

You do not need to create a 100% solution here. I am looking for you to demonstrate an understanding of the critical design flaw in this training set (particularly for a patch based, as opposed to segmentation, approach), and to show critical thinking in how you might fix it. This may include re-sampling the existing training data, including new training data, or changing the bands being used (or all of the above).



A6:

Alright, we have no “no solar panel” class. In order to create patches, I created a composite image over a predefined area of interest without clouds, like we did in the first few labs. Then I generated 500 random points (I regretted that number very quickly, but more later) and created 256x256 patches around them. I then sent a task to GEE to download all of these 500 patches to my drive. Well after half an hour I had around 100 patches, so I stopped the job. I then did data augmentation with these 100 patches. I rotated each one of them 3 times, and mirrored each rotation. That gave me 8x100=800 patches for my “no solar panel” class. that makes the dataset still be imbalanced, but I decided to be satisfied for the scope of this. Because, if the model would still only predict yes, it would now get to an accuracy of around 62%, and I think I can do better.

Great, I’m actually very happy that you made us create this visualization, because with the same settings, the two patches look very different. I will just normalize the entire dataset to make everything look the same. I got the following output when comparing the two patch sets:

It seems like I made a mistake when specifying the bands and the shape, or maybe these are the GEE defaults, I’m not sure. Fact is, this exercise shall just show our understanding and not deliver a perfect solution, so I won’t create new patches, but I’ll adjust both datasets to match eachother and become identical in their shape to be able to continue.
I then unified the datasets and kept trac of their labels, then i augmented the “no” class with rotating and mirroring, creating 8 patches out of each single one, and then normalized the whole dataset to have the pixels have all the same brightness etc.
Here is a quick visualisation after normalization showing that I’m trying but I’m not quite getting there yet… I tried to standardize using the average over the whole dataset.

After trying some stuff out and it did not work, I decided to move on.


**Q7**: Exercise: re-run the model with your new training data (remembering that I am not looking for a total solution), and further adjusted hyperparameters/RAM options. Write two paragraphs that:
1. Sets out **all the changes** you have made and the reasoning for them.
2. Asses the training and model results in terms of performance statistics, commenting on how/if your training data changes have made an impact.

You may include figures to support your analysis if you wish, remembering to present them to publication standard if you do so.

A7:

Alright, I’m onto retraining the model with the new dataset. Here are the changes that I made:

I increased the number of epochs from 5 to 25. More epochs means more opportunities to learn for the model, and 5 really is a ridiculous number, whereas 25 is also not way better. but more is more. I also increased the patience for the early stopper from 3 to 5 to give the model a bit more room (This turned out to be unnecessary because the training stagnated after 2 epochs already and had an accuracy of 100% !!!!!!!!!!!!!! :((((( ). Anyway, I added one more convolutional layer and increased the dense layer size to give the model a bit more capacity to learn. I ran the model a second time with an early stop waiting of back to 3 epochs, and it stopped after 3 epochs because the accuracy was again at 100%, which is very unsatisfying because obviously, I did something wrong along the way. I think the problem lies in the fact that I did not manage to normalize the dataset properly, and the patches from both classes have very significant brightness differences which makes the model overfit again, but for a different reason.


In [ ]:
# Set up GEE API
import ee
ee.Authenticate()
ee.Initialize(project='clara-geog761-tryout-1')

In [ ]:
# Import other libs
import geemap
import geopandas as gpd
import pandas as pd

# Define area of interest
aoi = ee.Geometry.Rectangle([175.3, -41, 175, -41.2])
Map = geemap.Map(center=[-41.3, 174.75], zoom=10)
Map.addLayer(aoi, {}, 'AOI')

# Visualize on the map to check got it right... (always a good idea)
Map

In [ ]:
# Set up your filter Sentinel-2 imagery
# A function that masks clouds in your S2 images via QA band
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000).select(['B2', 'B3', 'B4', 'B8'])

# Now build the clean collection with selected bands
s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
      .filterBounds(aoi) #This filters out all available images that overlap with my aoi
      .filterDate('2020-01-01', '2020-12-31')
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
      .map(mask_s2_clouds)
      .median() #<- think about this step and what it is doing to your image when you build your single composite from the image collection
      .clip(aoi)) #This takes the final composite image that is an overlap of diferent images with different boundarie, all somehow inlcuding my aoi, and cutting it to only have my aoi

In [ ]:
# Set up the visualisation and then once again check it has worked, stacking layers in a sensible order
Map = geemap.Map(center=[-41.1, 175.1], zoom=11) #<- reset the map object to clear out other layers from earlier cells
Map.addLayer(s2, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'S2 RGB')
Map.addLayer(aoi, {'color': 'red'}, 'AOI')
Map

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!pip install --upgrade earthengine-api

In [ ]:
BANDS_TO_EXPORT = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']

# Assume 's2' is your ee.ImageCollection and 'aoi' is your ee.Geometry
NUM_PATCHES_TO_CREATE = 500
PATCH_SIZE_METERS = 256 # Patch size in meters (e.g., 256x256 meters)

EXPORT_FOLDER = 'GEE_Patches_No_Solar' # A folder in your Google Drive

In [ ]:
# 2. Generate random points within the AOI
random_points = ee.FeatureCollection.randomPoints(
    region=aoi,
    points=NUM_PATCHES_TO_CREATE,
    seed=42 # Use a seed for reproducibility
)

# 3. Get the list of points from the server to the client (your Colab notebook)
# .toList() brings the collection over so we can loop through it.
points_list = random_points.toList(NUM_PATCHES_TO_CREATE)

print(f"Starting to create {points_list.size().getInfo()} export tasks...")

# 4. Loop through each point and create a separate export task for each one
for i in range(NUM_PATCHES_TO_CREATE):
    # Get a single point from the list
    point = ee.Feature(points_list.get(i))

    # Create the patch geometry on the server
    patch_geometry = point.geometry().buffer(PATCH_SIZE_METERS / 2).bounds()

    # Clip the source image to this patch's geometry
    patch_image = s2.clip(patch_geometry)

    # Define a unique name for each patch file
    file_name = f'no_solar_patch_{i:03d}' # e.g., no_solar_patch_001

    # Create and start the export task for this single patch
    task = ee.batch.Export.image.toDrive(
        image=patch_image,
        description=file_name,
        folder=EXPORT_FOLDER,
        fileNamePrefix=file_name,
        scale=10,
        fileFormat='GeoTIFF'
    )
    task.start()

print(f"\nAll {NUM_PATCHES_TO_CREATE} export tasks have been started.")
print(f"Check the 'Tasks' tab in the GEE Code Editor to monitor their progress.")
print(f"The files will be saved in your Google Drive folder: '{EXPORT_FOLDER}'")


In [ ]:
# DELETE TASK BECAUSE IT TAKES WAY TOO LONG FOR 500 PATCHES TO BE CREATED

import ee

# Make sure you have authenticated and initialized GEE
# ee.Authenticate()
# ee.Initialize()

# 1. Get a list of all your current tasks from the GEE server
tasks = ee.batch.Task.list()

cancelled_count = 0
print("Searching for tasks to cancel...")

# 2. Loop through the tasks
for task in tasks:
    # 3. Check if the task is part of your batch export and is still active
    if 'no_solar_patch' in task.config['description'] and task.state in ['RUNNING', 'READY']:
        # 4. If it matches, cancel it
        task.cancel()
        cancelled_count += 1
        print(f"Cancelling task: {task.config['description']}")

if cancelled_count > 0:
    print(f"\nSuccessfully sent cancellation requests for {cancelled_count} tasks.")
else:
    print("\nNo active 'no_solar_patch' tasks found to cancel.")

In [ ]:
# CREATING VISUALIZATION OF EXISTING PATCH VS CREATED PATCH BY MYSELF

import rasterio
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive

# Mount Google Drive to access the files
try:
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    print(f"Could not mount drive: {e}")

# --- File Paths ---
# Path to your original patch with a solar panel
old_patch_path = "/content/train_data/s2_image/train_s2_image_1.tif"

# Path to one of your newly created patches without a solar panel
new_patch_path = "/content/drive/MyDrive/GEE_Patches_No_Solar/no_solar_patch_001.tif"

# --- Visualization Function ---
def create_comparison_figure(path1, path2):
    """
    Reads two multi-band GeoTIFFs and displays them side-by-side
    as true-color images.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # --- Process and display the first image ---
    try:
        with rasterio.open(path1) as src:
            # Read true-color bands (B4=Red, B3=Green, B2=Blue)
            rgb1 = np.dstack((src.read(4), src.read(3), src.read(2)))
            # Contrast stretch for better visualization
            p2, p98 = np.percentile(rgb1, (2, 98))
            rgb1_stretched = np.clip((rgb1 - p2) * (255.0 / (p98 - p2)), 0, 255).astype(np.uint8)
            ax1.imshow(rgb1_stretched)
            ax1.set_title("Original Patch ('Yes' Class)", fontsize=12)
    except Exception as e:
        ax1.text(0.5, 0.5, f"Could not load:\n{os.path.basename(path1)}", ha='center', va='center')
        print(f"Error loading {path1}: {e}")

    # --- Process and display the second image ---
    try:
        with rasterio.open(path2) as src:
            rgb2 = np.dstack((src.read(4), src.read(3), src.read(2)))
            p2, p98 = np.percentile(rgb2, (2, 98))
            rgb2_stretched = np.clip((rgb2 - p2) * (255.0 / (p98 - p2)), 0, 255).astype(np.uint8)
            ax2.imshow(rgb2_stretched)
            ax2.set_title("Generated Patch ('No' Class)", fontsize=12)
    except Exception as e:
        ax2.text(0.5, 0.5, f"Could not load:\n{os.path.basename(path2)}", ha='center', va='center')
        print(f"Error loading {path2}: {e}")

    # --- Final Figure Details ---
    for ax in [ax1, ax2]:
        ax.axis('off')

    caption = "Side-by-side comparison of an original training patch containing solar panels (left) \nand a randomly generated patch for the 'no solar panel' class (right)."
    fig.suptitle(caption, y=0.08, fontsize=14)

    plt.tight_layout(rect=[0, 0.1, 1, 1]) # Adjust layout to make room for caption
    plt.show()

# --- Run the visualization ---
create_comparison_figure(old_patch_path, new_patch_path)

In [ ]:
# PATCH COMPARISON BETWEEN ORIGINAL TRAINING DATA SET AND MY GENERATED DATASET

import rasterio
import glob
import os

# --- Configuration ---
# 1. SET THIS to the path of your ORIGINAL 'yes' solar panel patches
YES_PATCH_DIR = "/content/train_data/s2_image/"

# 2. SET THIS to the path of your GENERATED 'no' solar panel patches
#    (The folder you unzipped after running the augmentation)
NO_PATCH_DIR = "/content/drive/MyDrive/GEE_Patches_No_Solar/"


def inspect_single_patch(patch_path):
    """
    Opens a single GeoTIFF and prints its key properties.
    """
    try:
        with rasterio.open(patch_path) as src:
            print(f"File: {os.path.basename(patch_path)}")
            print(f"  - Shape (bands, height, width): ({src.count}, {src.height}, {src.width})")
            print(f"  - Band Count: {src.count}")
            print(f"  - Data Type: {src.dtypes[0]}")
            print("-" * 40)
    except Exception as e:
        print(f"Could not inspect file: {patch_path}")
        print(f"Error: {e}")
        print("-" * 40)


# --- Run the Comparison ---
if __name__ == "__main__":
    print("="*50)
    print("     SINGLE PATCH COMPARISON REPORT")
    print("="*50 + "\n")

    # --- Inspect one 'YES' patch ---
    print("--- Inspecting one 'YES' Patch (Original Dataset) ---\n")
    yes_files = glob.glob(os.path.join(YES_PATCH_DIR, '*.tif'))
    if yes_files:
        inspect_single_patch(yes_files[0])
    else:
        print(f"No .tif files found in '{YES_PATCH_DIR}'")
        print("-" * 40)

    # --- Inspect one 'NO' patch ---
    print("\n--- Inspecting one 'NO' Patch (Generated Dataset) ---\n")
    no_files = glob.glob(os.path.join(NO_PATCH_DIR, '*.tif'))
    if no_files:
        inspect_single_patch(no_files[0])
    else:
        print(f"No .tif files found in '{NO_PATCH_DIR}'")
        print("-" * 40)

    print("\nFor the datasets to be compatible, their properties should be identical.")

In [ ]:
# DATEN ANGLEICHUNG, I NEED TO ADJUST BOTH DATASET TO BECOME IDENTICAL IN SHAPE

import rasterio
import numpy as np
import pandas as pd
import glob
import os
import shutil

# --- Configuration ---
# 1. Path to your ORIGINAL 'yes' solar panel patches (12 bands)
YES_PATCH_DIR = "/content/train_data/s2_image/"

# 2. Path to your GENERATED 'no' solar panel patches (4 bands, wrong size)
NO_PATCH_DIR = "/content/drive/MyDrive/GEE_Patches_No_Solar/" # Using the un-augmented GEE patches

# 3. Path for the NEW, UNIFIED dataset. This script will create it.
UNIFIED_OUTPUT_DIR = "/content/Unified_Dataset/"

# 4. Path for the final CSV manifest file.
FINAL_MANIFEST_PATH = "/content/unified_dataset_labels.csv"


def get_band_indices_to_keep(sample_yes_path, sample_no_path):
    """
    Inspects sample files to dynamically determine which band indices to keep.
    If band descriptions are not present, it falls back to a safe default.
    """
    print("--- Checking Band Information ---")
    try:
        with rasterio.open(sample_no_path) as src_no:
            target_bands = src_no.descriptions
            if not all(target_bands):
                raise ValueError("No band descriptions found in the 'no' patch.")

        with rasterio.open(sample_yes_path) as src_yes:
            source_bands = src_yes.descriptions
            if not all(source_bands):
                raise ValueError("No band descriptions found in the 'yes' patch.")

        print(f"  - Detected target bands from 'no' patch: {target_bands}")
        print(f"  - Found source bands from 'yes' patch: {source_bands}")

        # Find the 0-based index for each target band in the source list
        indices = [source_bands.index(band) for band in target_bands]
        print(f"  - Dynamically determined band indices to keep: {indices}")
        return indices

    except (ValueError, AttributeError, IndexError) as e:
        print(f"  - Warning: Could not dynamically determine bands. Reason: {e}")
        print("  - Falling back to default Sentinel-2 bands (B2, B3, B4, NIR).")
        # Default 0-based indices for B2, B3, B4, B8
        return [1, 2, 3, 7]


def unify_datasets(yes_dir, no_dir, output_dir):
    """
    Unifies two mismatched patch datasets by clipping bands and cropping dimensions.
    """
    # Create a clean output directory
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)
    print(f"Created clean output directory: {output_dir}")

    all_file_info = []

    # Get file lists
    yes_files = glob.glob(os.path.join(yes_dir, '*.tif'))
    no_files = glob.glob(os.path.join(no_dir, '*.tif'))

    if not yes_files or not no_files:
        print("\nError: Could not find .tif files in one or both directories. Aborting.")
        return pd.DataFrame()

    # --- Dynamically determine which bands to keep ---
    BANDS_TO_KEEP = get_band_indices_to_keep(yes_files[0], no_files[0])

    # --- 1. Process 'YES' patches (Clip bands) ---
    print(f"\nProcessing {len(yes_files)} 'yes' patches (clipping to {len(BANDS_TO_KEEP)} bands)...")

    for path in yes_files:
        with rasterio.open(path) as src:
            if src.count < max(BANDS_TO_KEEP) + 1:
                print(f"  - Warning: Skipping {os.path.basename(path)}, not enough bands.")
                continue

            all_bands_data = src.read()
            clipped_data = all_bands_data[BANDS_TO_KEEP, :, :]

            profile = src.profile
            profile.update(count=len(BANDS_TO_KEEP), driver='GTiff', dtype=clipped_data.dtype)

            new_filename = os.path.basename(path)
            new_path = os.path.join(output_dir, new_filename)
            with rasterio.open(new_path, 'w', **profile) as dst:
                dst.write(clipped_data)

            all_file_info.append({'filepath': new_path, 'label': 1})

    # --- 2. Process 'NO' patches (Center crop) ---
    print(f"\nProcessing {len(no_files)} 'no' patches (cropping to 23x23)...")

    TARGET_HEIGHT, TARGET_WIDTH = 23, 23

    for path in no_files:
        with rasterio.open(path) as src:
            data = src.read()

            h, w = src.height, src.width
            top = (h - TARGET_HEIGHT) // 2
            left = (w - TARGET_WIDTH) // 2

            cropped_data = data[:, top:top + TARGET_HEIGHT, left:left + TARGET_WIDTH]

            profile = src.profile
            profile.update(height=TARGET_HEIGHT, width=TARGET_WIDTH, dtype=cropped_data.dtype)

            new_filename = os.path.basename(path)
            new_path = os.path.join(output_dir, new_filename)
            with rasterio.open(new_path, 'w', **profile) as dst:
                dst.write(cropped_data)

            all_file_info.append({'filepath': new_path, 'label': 0})

    print("\nDataset unification complete.")
    return pd.DataFrame(all_file_info)

# --- Run the Unification ---
if __name__ == "__main__":
    manifest_df = unify_datasets(YES_PATCH_DIR, NO_PATCH_DIR, UNIFIED_OUTPUT_DIR)

    if not manifest_df.empty:
        manifest_df.to_csv(FINAL_MANIFEST_PATH, index=False)
        print(f"\nFinal manifest file saved to: {FINAL_MANIFEST_PATH}")
        print(f"Total unified patches: {len(manifest_df)}")
        print("\nYour dataset is now unified and ready for normalization or training!")
    else:
        print("\nNo files were processed. The final manifest was not created.")

In [ ]:
# DATASET VERYFIER

import rasterio
import numpy as np
import pandas as pd
import glob
import os
import shutil

# --- Configuration ---
# 1. Path to your ORIGINAL 'yes' solar panel patches (inconsistent sizes)
YES_PATCH_DIR = "/content/train_data/s2_image/"

# 2. Path to your GENERATED 'no' solar panel patches (4 bands, wrong size)
NO_PATCH_DIR = "/content/drive/MyDrive/GEE_Patches_No_Solar/"

# 3. Path for the FINAL, UNIFIED dataset. This script will create it.
UNIFIED_OUTPUT_DIR = "/content/Unified_Dataset_Final/"

# 4. Path for the final CSV manifest file.
FINAL_MANIFEST_PATH = "/content/unified_dataset_labels_final.csv"

# 5. Define the one true shape for all patches
TARGET_BANDS = 4
TARGET_HEIGHT = 23
TARGET_WIDTH = 23


def get_band_indices_to_keep(sample_yes_path, sample_no_path):
    """
    Inspects sample files to dynamically determine which band indices to keep.
    """
    print("--- Checking Band Information ---")
    try:
        with rasterio.open(sample_no_path) as src_no:
            target_bands = src_no.descriptions
            if not all(target_bands): raise ValueError("No band descriptions in 'no' patch.")
        with rasterio.open(sample_yes_path) as src_yes:
            source_bands = src_yes.descriptions
            if not all(source_bands): raise ValueError("No band descriptions in 'yes' patch.")

        print(f"  - Target bands from 'no' patch: {target_bands}")
        indices = [source_bands.index(band) for band in target_bands]
        indices_1based = [i + 1 for i in indices]
        print(f"  - Dynamically determined band indices to keep: {indices_1based}")
        return indices_1based
    except Exception as e:
        print(f"  - Warning: Could not dynamically determine bands. Reason: {e}")
        print("  - Falling back to default Sentinel-2 bands (B2, B3, B4, B8).")
        return [2, 3, 4, 8]


def center_crop_data(data, target_height, target_width):
    """
    Crops a numpy array (bands, height, width) from the center.
    """
    _ , h, w = data.shape
    start_x = w // 2 - target_width // 2
    start_y = h // 2 - target_height // 2
    return data[:, start_y:start_y + target_height, start_x:start_x + target_width]


def unify_datasets_final(yes_dir, no_dir, output_dir):
    """
    Unifies two mismatched datasets by forcing all patches to a target
    dimension using band clipping and center cropping.
    """
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)
    print(f"Created clean output directory: {output_dir}")

    all_file_info = []
    yes_files = glob.glob(os.path.join(yes_dir, '*.tif'))
    no_files = glob.glob(os.path.join(no_dir, '*.tif'))

    if not yes_files or not no_files:
        print("\nError: Could not find .tif files. Please check YES_PATCH_DIR and NO_PATCH_DIR.")
        return pd.DataFrame()

    BANDS_TO_KEEP = get_band_indices_to_keep(yes_files[0], no_files[0])

    # --- 1. Process 'YES' patches (Clip bands AND Center Crop) ---
    print(f"\nProcessing {len(yes_files)} 'yes' patches (clipping bands and cropping to 23x23)...")

    for path in yes_files:
        with rasterio.open(path) as src:
            profile = src.profile
            # Read only the bands we need
            clipped_data = src.read(BANDS_TO_KEEP)

            # Perform a center crop on the clipped data
            cropped_data = center_crop_data(clipped_data, TARGET_HEIGHT, TARGET_WIDTH)

            # Check if cropping was successful
            if cropped_data.shape != (TARGET_BANDS, TARGET_HEIGHT, TARGET_WIDTH):
                print(f"  - Warning: Skipping {os.path.basename(path)} due to incompatible size for cropping.")
                continue

            profile.update(count=TARGET_BANDS, height=TARGET_HEIGHT, width=TARGET_WIDTH, dtype=cropped_data.dtype)

            new_path = os.path.join(output_dir, os.path.basename(path))
            with rasterio.open(new_path, 'w', **profile) as dst:
                dst.write(cropped_data)

            all_file_info.append({'filepath': new_path, 'label': 1})

    # --- 2. Process 'NO' patches (Center Crop only) ---
    print(f"\nProcessing {len(no_files)} 'no' patches (cropping to 23x23)...")

    for path in no_files:
        with rasterio.open(path) as src:
            profile = src.profile
            # Read all 4 bands
            data = src.read()

            # Perform a center crop
            cropped_data = center_crop_data(data, TARGET_HEIGHT, TARGET_WIDTH)

            if cropped_data.shape != (TARGET_BANDS, TARGET_HEIGHT, TARGET_WIDTH):
                print(f"  - Warning: Skipping {os.path.basename(path)} due to incompatible size for cropping.")
                continue

            profile.update(height=TARGET_HEIGHT, width=TARGET_WIDTH, dtype=cropped_data.dtype)

            new_path = os.path.join(output_dir, os.path.basename(path))
            with rasterio.open(new_path, 'w', **profile) as dst:
                dst.write(cropped_data)

            all_file_info.append({'filepath': new_path, 'label': 0})

    print("\nDataset unification complete.")
    return pd.DataFrame(all_file_info)

# --- Run the Unification ---
if __name__ == "__main__":
    manifest_df = unify_datasets_final(YES_PATCH_DIR, NO_PATCH_DIR, UNIFIED_OUTPUT_DIR)

    if not manifest_df.empty:
        manifest_df.to_csv(FINAL_MANIFEST_PATH, index=False)
        print(f"\nFinal manifest file saved to: {FINAL_MANIFEST_PATH}")
        print(f"Total unified patches: {len(manifest_df)}")
    else:
        print("\nNo files were processed.")


In [ ]:
# DATA AUGMENTATION OF THE NO CLASS IN THE ALREADY UNIFIED AND CROPPED DATASET

import rasterio
import numpy as np
import glob
import os
import shutil
import pandas as pd
from tqdm import tqdm

# --- Configuration ---
# 1. Path to your UNIFIED dataset (the output from the previous step)
UNIFIED_DATA_DIR = "/content/Unified_Dataset_Final/"

# 2. Path for the FINAL dataset CSV manifest (created by the unifier script)
MANIFEST_PATH = "/content/unified_dataset_labels_final.csv"

# 3. Path for the NEW augmented 'no' patches. This script will create it.
AUGMENTED_OUTPUT_DIR = "/content/Augmented_Unified_No_Solar/"

# 4. Path for the FINAL CSV manifest that includes the augmented files.
FINAL_AUGMENTED_MANIFEST_PATH = "/content/final_augmented_dataset_labels.csv"


def augment_unified_patches(unified_dir, manifest_path, output_dir):
    """
    Reads unified 'no' patches, creates 8 augmented versions of each,
    and saves them to a new directory. Then creates a new manifest file
    that includes both the 'yes' patches and the new augmented 'no' patches.
    """
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)
    print(f"Created clean output directory: {output_dir}")

    try:
        manifest_df = pd.read_csv(manifest_path)
    except FileNotFoundError:
        print(f"Error: Manifest file not found at {manifest_path}. Aborting.")
        return

    # Separate the 'yes' and 'no' patches from the manifest
    yes_patches_df = manifest_df[manifest_df['label'] == 1]
    no_patches_df = manifest_df[manifest_df['label'] == 0]

    no_files_to_augment = no_patches_df['filepath'].tolist()

    print(f"Found {len(no_files_to_augment)} unified 'no' patches to augment...")

    new_augmented_rows = []

    for path in tqdm(no_files_to_augment, desc="Augmenting 'No' Patches"):
        try:
            with rasterio.open(path) as src:
                data = src.read()
                profile = src.profile

                base_filename = os.path.splitext(os.path.basename(path))[0]

                # Define the 8 transformations
                transforms = {
                    'orig': data,
                    'rot90': np.rot90(data, 1, axes=(1, 2)),
                    'rot180': np.rot90(data, 2, axes=(1, 2)),
                    'rot270': np.rot90(data, 3, axes=(1, 2))
                }

                # Apply transformations and save
                for name, transformed_data in transforms.items():
                    # Save the rotated version
                    out_path = os.path.join(output_dir, f"{base_filename}_{name}.tif")
                    with rasterio.open(out_path, 'w', **profile) as dst:
                        dst.write(transformed_data)
                    new_augmented_rows.append({'filepath': out_path, 'label': 0})

                    # Save the flipped version of the rotation
                    flipped_data = np.flip(transformed_data, axis=2)
                    out_path_flipped = os.path.join(output_dir, f"{base_filename}_{name}_flip.tif")
                    with rasterio.open(out_path_flipped, 'w', **profile) as dst:
                        dst.write(flipped_data)
                    new_augmented_rows.append({'filepath': out_path_flipped, 'label': 0})

        except Exception as e:
            print(f"  - Warning: Could not process {os.path.basename(path)}. Skipping. Error: {e}")

    augmented_no_df = pd.DataFrame(new_augmented_rows)

    # Combine the original 'yes' patches with the new augmented 'no' patches
    final_df = pd.concat([yes_patches_df, augmented_no_df], ignore_index=True)

    print(f"\nAugmentation complete.")
    print(f"  - Original 'yes' patches: {len(yes_patches_df)}")
    print(f"  - New augmented 'no' patches: {len(augmented_no_df)}")
    print(f"  - Total patches in new dataset: {len(final_df)}")

    return final_df

# --- Run the Augmentation ---
if __name__ == "__main__":
    final_manifest = augment_unified_patches(UNIFIED_DATA_DIR, MANIFEST_PATH, AUGMENTED_OUTPUT_DIR)

    if final_manifest is not None and not final_manifest.empty:
        final_manifest.to_csv(FINAL_AUGMENTED_MANIFEST_PATH, index=False)
        print(f"\nFinal augmented manifest file saved to: {FINAL_AUGMENTED_MANIFEST_PATH}")


In [ ]:
# NORMALIZE ENTIRE DATASET
# This normlization DOES NOT WORK, go see the post-normalization verification for some funny end results.
# But oh well, at least I know how to create patches and am halfway to being able to create a usable dataset. It's definitely not usable yet.

import rasterio
import numpy as np
import pandas as pd
import os
import shutil
from tqdm import tqdm

# --- Configuration ---
# 1. Path to the FINAL CSV manifest from the augmentation step.
AUGMENTED_MANIFEST_PATH = "/content/final_augmented_dataset_labels.csv"

# 2. Directory where the unified TIFF patches are stored.
#    This is needed to read the files for calculating statistics.
UNIFIED_PATCH_DIR = "/content/Unified_Dataset_Final/"

# 3. Directory to save the final, normalized dataset (as .npy files).
OUTPUT_NORMALIZED_DIR = "/content/Final_Normalized_Dataset/"

# 4. Path to save the final CSV manifest pointing to the .npy files.
FINAL_NORMALIZED_MANIFEST_PATH = "/content/final_normalized_dataset_manifest.csv"

# 5. Path to save the statistics file.
STATS_FILE_PATH = "/content/dataset_stats.npz"


def calculate_statistics_from_all(base_dir, all_files):
    """
    Calculates the mean and standard deviation for each band from the
    ENTIRE dataset.
    """
    print(f"Calculating global statistics from {len(all_files)} patches...")

    band_pixel_values = [[] for _ in range(4)] # Assuming 4 bands

    for file_path in tqdm(all_files, desc="Calculating Stats"):
        try:
            # The manifest contains full paths, so we use them directly
            with rasterio.open(file_path) as src:
                data = src.read()
                for i in range(data.shape[0]):
                    band_pixel_values[i].extend(data[i].flatten())
        except Exception as e:
            print(f"  - Warning: Could not read {os.path.basename(file_path)} for stats. Skipping. Error: {e}")

    mean_per_band = np.array([np.mean(pixels) for pixels in band_pixel_values])
    std_per_band = np.array([np.std(pixels) for pixels in band_pixel_values])

    mean = mean_per_band[:, np.newaxis, np.newaxis]
    std = std_per_band[:, np.newaxis, np.newaxis]

    std[std == 0] = 1 # Avoid division by zero

    return mean, std


def normalize_dataset(df, output_dir, mean, std_dev):
    """
    Normalizes all patches in the dataframe using pre-calculated stats
    and saves them as .npy files.
    """
    new_rows = []

    for index, row in tqdm(df.iterrows(), total=len(df), desc="Normalizing Patches"):
        patch_path = row['filepath']
        label = row['label']

        try:
            # We need to read from the original unified/augmented TIFFs, not the final .npy files
            with rasterio.open(patch_path) as src:
                data = src.read().astype(np.float32)

                normalized_data = (data - mean) / std_dev

                base_filename = os.path.splitext(os.path.basename(patch_path))[0]
                new_filename = f"{base_filename}_norm.npy"
                new_filepath = os.path.join(output_dir, new_filename)

                np.save(new_filepath, normalized_data)

                new_rows.append({'filepath': new_filepath, 'label': label})

        except Exception as e:
            print(f"  - Warning: Could not normalize {os.path.basename(patch_path)}. Skipping. Error: {e}")

    return pd.DataFrame(new_rows)


# --- Main Execution Block ---
if __name__ == "__main__":
    if os.path.exists(OUTPUT_NORMALIZED_DIR):
        shutil.rmtree(OUTPUT_NORMALIZED_DIR)
    os.makedirs(OUTPUT_NORMALIZED_DIR)
    print(f"Created clean output directory: {OUTPUT_NORMALIZED_DIR}")

    try:
        manifest_df = pd.read_csv(AUGMENTED_MANIFEST_PATH)
    except FileNotFoundError:
        print(f"❌ ERROR: Augmented manifest file not found at {AUGMENTED_MANIFEST_PATH}. Aborting.")
        exit()

    # --- 1. Calculate Statistics from the ENTIRE augmented dataset ---
    # Get a list of all .tif file paths from the manifest
    all_tiff_files = manifest_df['filepath'].tolist()

    # Pass the full list to the statistics function
    mean, std = calculate_statistics_from_all(UNIFIED_PATCH_DIR, all_tiff_files)

    print("\nGlobal statistics calculation complete.")
    np.savez(STATS_FILE_PATH, mean=mean, std=std)
    print(f"Statistics saved to {STATS_FILE_PATH}")

    # --- 2. Normalize the ENTIRE dataset using these global stats ---
    final_normalized_df = normalize_dataset(manifest_df, OUTPUT_NORMALIZED_DIR, mean, std)

    # --- 3. Save the FINAL normalized manifest CSV ---
    if not final_normalized_df.empty:
        final_normalized_df.to_csv(FINAL_NORMALIZED_MANIFEST_PATH, index=False)
        print(f"\n✅ SUCCESS: Normalization complete.")
        print(f"Final model-ready manifest saved to: {FINAL_NORMALIZED_MANIFEST_PATH}")
    else:
        print("\n❌ FAILED: No patches were normalized. Check for warnings.")



In [ ]:
# post normalization verification step

import numpy as np
import pandas as pd
import os
import rasterio
import matplotlib.pyplot as plt

# --- Configuration ---
# 1. Path to the AUGMENTED (but not normalized) manifest CSV. THIS PATH HAS BEEN UPDATED.
AUGMENTED_MANIFEST_PATH = "/content/final_augmented_dataset_labels.csv"

# 2. Path to the FINAL NORMALIZED manifest CSV.
NORMALIZED_MANIFEST_PATH = "/content/final_normalized_dataset_manifest.csv"

# 3. Path to the saved statistics file.
STATS_FILE_PATH = "/content/dataset_stats.npz"


def find_original_base_name(augmented_filename):
    """
    Strips all augmentation and normalization suffixes to find the
    original base filename from the unified dataset.
    e.g., 'no_solar_patch_040_rot180_flip_norm.npy' -> 'no_solar_patch_040.tif'
    """
    base = augmented_filename.replace('_norm.npy', '')
    suffixes = ['_orig_flip', '_rot90_flip', '_rot180_flip', '_rot270_flip',
                '_orig', '_rot90', '_rot180', '_rot270', '_flip']
    for suffix in suffixes:
        if base.endswith(suffix):
            return base[:-len(suffix)] + '.tif'
    return base + '.tif'


def verify_normalization(augmented_manifest_path, normalized_manifest_path, stats_path):
    """
    Verifies the normalization by checking statistics and visualizing a before-and-after comparison.
    """
    print("==================================================")
    print("      POST-NORMALIZATION VERIFICATION SCRIPT")
    print("==================================================")

    try:
        augmented_df = pd.read_csv(augmented_manifest_path)
        normalized_df = pd.read_csv(normalized_manifest_path)
    except FileNotFoundError as e:
        print(f"❌ FAILED: Could not find a required file. Error: {e}")
        return

    print("Loaded all necessary files successfully.\n")
    print("--- Statistical Verification ---")
    print("Checking if the mean of normalized patches is ~0.0 and std dev is ~1.0...")

    # Select one random 'yes' and one random 'no' patch from the normalized set
    yes_sample_norm = normalized_df[normalized_df['label'] == 1].sample(1).iloc[0]
    no_sample_norm = normalized_df[normalized_df['label'] == 0].sample(1).iloc[0]

    # --- Verify the 'yes' sample ---
    yes_data_norm = np.load(yes_sample_norm['filepath'])
    print(f"\n- YES Patch ({os.path.basename(yes_sample_norm['filepath'])}):")
    print(f"  - Mean: {np.mean(yes_data_norm):.4f} (should be close to 0)")
    print(f"  - Std Dev: {np.std(yes_data_norm):.4f} (should be close to 1)")

    # --- Verify the 'no' sample ---
    no_data_norm = np.load(no_sample_norm['filepath'])
    print(f"\n- NO Patch ({os.path.basename(no_sample_norm['filepath'])}):")
    print(f"  - Mean: {np.mean(no_data_norm):.4f} (should be close to 0)")
    print(f"  - Std Dev: {np.std(no_data_norm):.4f} (should be close to 1)")

    print("\n--- Visual Verification ---")
    print("Displaying a before-and-after comparison...")

    # Get the base filename of the normalized samples, without the suffix or directory
    yes_filename_before = os.path.basename(yes_sample_norm['filepath']).replace('_norm.npy', '.tif')
    no_filename_before = os.path.basename(no_sample_norm['filepath']).replace('_norm.npy', '.tif')

    # Find the full paths in the augmented dataframe by searching for the base filename
    yes_path_before_series = augmented_df[augmented_df['filepath'].str.endswith(yes_filename_before, na=False)]
    no_path_before_series = augmented_df[augmented_df['filepath'].str.endswith(no_filename_before, na=False)]

    # Check that we found a match before trying to access it
    if yes_path_before_series.empty or no_path_before_series.empty:
        print(f"\n❌ FAILED: Could not find the original 'before' patch for visualization.")
        print(f"Searched for filenames ending with '{yes_filename_before}' and '{no_filename_before}'.")
        return

    yes_path_before = yes_path_before_series['filepath'].iloc[0]
    no_path_before = no_path_before_series['filepath'].iloc[0]

    with rasterio.open(yes_path_before) as src:
        yes_data_before = src.read()
    with rasterio.open(no_path_before) as src:
        no_data_before = src.read()

    # Create a 2x2 plot
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))

    def visualize_rgb(data, ax, title):
        rgb = data[:3, :, :].transpose(1, 2, 0)
        p2, p98 = np.percentile(rgb, (2, 98))
        rgb_stretched = np.clip((rgb - p2) / (p98 - p2), 0, 1)
        ax.imshow(rgb_stretched)
        ax.set_title(title, fontsize=12)
        ax.axis('off')

    def visualize_norm(data, ax, title):
        rgb_norm = data[:3, :, :].transpose(1, 2, 0)
        rgb_norm_display = (rgb_norm + 3) / 6
        ax.imshow(np.clip(rgb_norm_display, 0, 1))
        ax.set_title(title, fontsize=12)
        ax.axis('off')

    visualize_rgb(yes_data_before, axes[0, 0], "'Yes' Patch (Before Normalization)")
    visualize_rgb(no_data_before, axes[0, 1], "'No' Patch (Before Normalization)")
    visualize_norm(yes_data_norm, axes[1, 0], "'Yes' Patch (After Normalization)")
    visualize_norm(no_data_norm, axes[1, 1], "'No' Patch (After Normalization)")

    fig.tight_layout()
    plt.show()


# --- Run the Verification ---
if __name__ == "__main__":
    verify_normalization(AUGMENTED_MANIFEST_PATH, NORMALIZED_MANIFEST_PATH, STATS_FILE_PATH)



In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from pathlib import Path
import os
import glob
import rasterio
from tqdm import tqdm
import cv2

# --- Configuration & Setup ---
# Set our random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# --- NEW DATA LOADING SECTION ---
try:
    MANIFEST_PATH = "/content/final_augmented_dataset_labels.csv"
    manifest_df = pd.read_csv(MANIFEST_PATH)
    print(f"Successfully loaded dataset manifest with {len(manifest_df)} entries.")
except FileNotFoundError:
    print(f"❌ ERROR: Could not find the manifest file at {MANIFEST_PATH}")
    print("Please ensure the augmentation script has been run successfully.")
    exit()

# --- BATCH PREPROCESSING ---
# Set storage folder
OUTPUT_DIR = "patch_store"
os.makedirs(OUTPUT_DIR, exist_ok=True)

TARGET_HEIGHT, TARGET_WIDTH = 256, 256
BATCH_SIZE = 100

X_batch, Y_batch = [], []
batch_idx = 0

# --- CRUCIAL FIX: Shuffle the manifest before creating batches ---
# This ensures that each .npz file contains a random mix of 'yes' and 'no' patches.
manifest_df = manifest_df.sample(frac=1, random_state=SEED).reset_index(drop=True)
print("Shuffled the dataset manifest to ensure mixed batches.")


# The main loop now iterates through our SHUFFLED manifest DataFrame
for index, row in tqdm(manifest_df.iterrows(), total=len(manifest_df), desc="Processing Patches"):
    patch_path = row['filepath']
    label = row['label']

    with rasterio.open(patch_path) as src:
        img = src.read().astype(np.float32)
        img = np.transpose(img, (1, 2, 0))

    img = cv2.resize(
        img,
        (TARGET_WIDTH, TARGET_HEIGHT),
        interpolation=cv2.INTER_LINEAR
    )
    img = img / 10000.0

    X_batch.append(img)
    Y_batch.append(label)

    if len(X_batch) >= BATCH_SIZE:
        X_batch_np = np.stack(X_batch)
        Y_batch_np = np.array(Y_batch)
        np.savez_compressed(
            os.path.join(OUTPUT_DIR, f"batch_{batch_idx:04d}.npz"),
            X=X_batch_np, Y=Y_batch_np
        )
        batch_idx += 1
        X_batch, Y_batch = [], []

if len(X_batch) > 0:
    X_batch_np = np.stack(X_batch)
    Y_batch_np = np.array(Y_batch)
    np.savez_compressed(
        os.path.join(OUTPUT_DIR, f"batch_{batch_idx:04d}.npz"),
        X=X_batch_np, Y=Y_batch_np
    )

print(f"\nPreprocessing complete. Saved {batch_idx + 1} batches to '{OUTPUT_DIR}'.")


# --- TF DATASET PREPARATION ---
all_files = sorted(glob.glob("patch_store/*.npz"))
np.random.shuffle(all_files)

train_split_index = int(len(all_files) * 0.8)
train_files = all_files[:train_split_index]
val_files = all_files[train_split_index:]

print(f"\nTotal batches: {len(all_files)}")
print(f"Training batches: {len(train_files)}")
print(f"Validation batches: {len(val_files)}")


def npz_loader_cls(path):
    with np.load(path.numpy().decode("utf-8")) as data:
        X = data["X"].astype(np.float32)
        Y = data["Y"].astype(np.float32)
    return X, Y

def tf_wrapper(path):
    X, Y = tf.py_function(npz_loader_cls, [path], [tf.float32, tf.float32])
    X.set_shape([None, TARGET_HEIGHT, TARGET_WIDTH, 4])
    Y.set_shape([None])
    return tf.data.Dataset.from_tensor_slices((X, Y))

def make_cls_dataset(file_list, batch_size, shuffle=True, repeat=True):
    files = tf.data.Dataset.from_tensor_slices(file_list)
    if shuffle:
        files = files.shuffle(len(file_list))
    ds = files.interleave(
        lambda f: tf_wrapper(f),
        cycle_length=4,
        num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(2048)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    if repeat:
        ds = ds.repeat()
    return ds

MODEL_BATCH_SIZE = 32
train_ds = make_cls_dataset(train_files, batch_size=MODEL_BATCH_SIZE, shuffle=True)
val_ds = make_cls_dataset(val_files, batch_size=MODEL_BATCH_SIZE, shuffle=False, repeat=False)


# --- MODEL DEFINITION & TRAINING ---
for X_sample, _ in train_ds.take(1):
    input_shape = X_sample.shape[1:]
    break

inputs = layers.Input(shape=input_shape)
x = layers.Conv2D(16, 3, activation='relu', padding='same')(inputs)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(64, activation='relu')(x)
# x = layers.Dropout(0.5)(x) # Dropout layer removed as requested
outputs = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs, outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

def count_samples(files):
    total = 0
    for f in files:
        total += np.load(f)["X"].shape[0]
    return total

n_train = count_samples(train_files)
n_val = count_samples(val_files)

steps_per_epoch = max(1, n_train // MODEL_BATCH_SIZE)
validation_steps = max(1, n_val // MODEL_BATCH_SIZE)

# --- UPDATED CALLBACKS ---
callbacks = [
    EarlyStopping(
        monitor="val_accuracy",  # CHANGE 1: Monitor validation accuracy
        patience=3,              # CHANGE 2: Stop after 3 epochs with no improvement
        restore_best_weights=True
    )]

# --- Run the model ---
history = model.fit(
    train_ds,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
    validation_steps=validation_steps,
    epochs=25,
    callbacks=callbacks
)



In [ ]:
import matplotlib.pyplot as plt

# --- Ensure you have the 'history' object from model.fit() ---

# --- Create the Plot ---
fig = plt.figure(figsize=(12, 5))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label="Training Loss")
plt.plot(history.history['val_loss'], label="Validation Loss")
plt.title("Model Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label="Training Accuracy")
plt.plot(history.history['val_accuracy'], label="Validation Accuracy")
plt.title("Model Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

# --- Title and Caption ---
fig.suptitle("Figure 1: Training Performance for the Enhanced Dataset", fontsize=16, weight='bold')

caption = (
    "Model performance on the enhanced dataset, which was balanced by creating and augmenting a 'no solar panel' class. "
    "The validation accuracy immediately reached 100%, suggesting the model may be overfitting or exploiting simple differences between the classes. "
    "Training was halted after 3 epochs by the EarlyStopping callback as no further improvement was observed."
)

# Added wrap=True to format the caption text into a paragraph
fig.text(0.5, -0.02, caption, ha='center', fontsize=12, wrap=True)

plt.tight_layout(rect=[0, 0.1, 1, 0.96]) # Adjust for suptitle
plt.show()



In [ ]:
def show_patch_predictions_grid(dataset, model, n_rows=2, n_cols=5):
    """
    Display predictions in a grid: n_rows x n_cols
    Includes histogram stretch for better visualization of RGB patches.
    """
    def stretch(img):
        """Contrast stretch to 0–1 for display."""
        img = img.astype(np.float32)
        p2, p98 = np.percentile(img, (2, 98))
        img = np.clip((img - p2) / (p98 - p2 + 1e-6), 0, 1)
        return img

    for X, Y_true in dataset.take(1):
        # live classification of the patch using the trained model
        Y_pred = model.predict(X, verbose=0)
        n_total = min(n_rows * n_cols, X.shape[0])

        fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*3, n_rows*3))
        axes = axes.flatten()

        for i in range(n_total):
            rgb = X[i, :, :, :3].numpy()
            rgb = stretch(rgb)  # apply histogram stretch
            axes[i].imshow(rgb)
            axes[i].set_title(f"T:{int(Y_true[i].numpy())}\nP:{Y_pred[i,0]:.2f}")
            axes[i].axis("off")

        # hide any unused axes
        for i in range(n_total, len(axes)):
            axes[i].axis("off")

        plt.tight_layout()
        plt.show()

# Example
show_patch_predictions_grid(val_ds, model, n_rows=1, n_cols=5)